In [1]:
# 구글 드라이브 마운트(cjyjob1993@gmail.com)
from google.colab import drive
drive.mount('/content/drive')

# lib 디렉토리를을 환경 변수에 추가
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/myCode/lib')

Mounted at /content/drive


# 신경망 학습

## 신경망 학습 과정 

1. 입력된 데이터가 각 층을 통과하며, 가중합(weight, bias) 및 활성화 함수 연산 반복
2. 출력층에서 계산된 값 출력
3. 예측값과 실제값(from 손실 함수)의 차이 계산
4. 경사하강법, 역전파를 통한 가중치 갱신
5. 학습 종료 까지 1~4 반복

## iteration

+ 가중치가 갱신되는 횟수.
+ 신경망 학습과정 1~4 반복 횟수(순전파 : 1,2 / 손실 함수 : 3 / 역전파 : 4)

## 순전파 (Forward Propagation)
입력층에서 입력된 신호가 각 층의 연산을 거쳐, 출력층에서 값으로 나오는 과정

1. 입력층(or 이전 은닉층)의 신호 수령
2. 가중합(가중치, 편향) 연산
3. 가중합 결과가 활성화 함수를 통과해 다음 층으로 전달

## 손실 함수 (Loss Function)
출력값과 타겟값의 차이를 구하는 함수
### MSE Mean-Squared Error
회귀 문제에 사용
### MAE Mean-Average Error
회귀 문제에 사용
### binary_crossentropy
이진 분류 문제에 사용
### CEE Cross-Entropy Error
다중 분류 문제에 사용
#### categorical_crossentropy
클래스간 비율이 유사 할 때 사용
#### sparse_categorical_crossentropy
클래스간 비율이 불균형 할 때 사용

## 역전파 (Backward Propagation)
출력층에서 입력층 방향으로, 경사하강법을 사용해, 손실을 줄이도록 weights와 bias를 갱신하는 과정. 이 과정에서 편미분과 Chain rule(연쇄 법칙)이 사용됨.

## 경사하강법 (Gradient Descent)
+ 손실 함수를 줄이는 방향으로
+ 손실 함수의 기울기가 작아지는 방향으로
+ 각 가중치 및 편향에 대한 손실 함수의 도함수(손실 함수를 편미분한 함수)를 계산해
+ 가중치 및 편향을 갱신하는 방법
+ 갱신된 가중치 = 갱신 전 가중치 - (학습률 * 해당 지점의 기울기)

## 옵티마이저 (Optimizer)
+ 경사를 내려가는 방법
+ 대표적인 것들은 아래와 같음
### GD (Gradient Descent, 경사하강법)
+ 일반적인 경사하강법
+ 모든 입력데이터를 사용해 가중치를 업데이트
### SGD (Stochastic Gradient Descent, 확률적 경사하강법)
+ 데이터의 양이 많아지면서, 기존의 GD는 학습 소요 시간이 지나치게 길어지면서 등장
+ 하나의 입력데이터만 사용해 가중치를 업데이트
+ 사용하는 데이터가 감소한 만큼, 소요 시간이 짧음
+ 이상치에 민감하게 작동함(불안정)
+ 1 epoch 에 하나의 데이터만 사용(다른 것들은 1 epoch에서 모든 데이터를 사용)
### 미니 배치 경사 하강법
+ SGD를 변형해 사용
+ N개의 데이터를 사용해 가중치를 업데이트
+ 속도도 개선되면서, 학습 안정성도 어느정도 유지
#### 배치 사이즈(batch size)
+ 미니 배치의 크기.
+ 일반적으로 2의 배수로 설정
+ 메모리가 허락하는 선에서 큰 사이즈가 학습에 안정적
### adam
+ 가장 안정적이고, 대중적인 옵티마이저

# 신경망 구현하기

In [97]:
# 라이브러리 임포트
import numpy as np
# 커스텀 라이브러리 임포트
from func_debug_log import debug
import numpy_fixSeed

In [98]:
# debug_flag 설정(0:미출력, 1:구현 함수 내부 확인 로그 미출력 ,2:모두 출력)
debug_flag = 1

In [99]:
class NN0 :
  def __init__(self):
    if debug_flag in [1, 2] : debug('신경망 초기화', __name__)

## 신경망 초기 상태 설정
weight, bias 초기화(랜덤값)

In [100]:
class NN1(NN0) :

  def __init__(self) :
    super().__init__()
    """
    신경망의 구조를 결정합니다.

    inputs : 입력층 노드 수
    hiddenNodes : 은닉층 노드 수
    outputNodes : 출력층 노드 수
    w1, w2 : layer 1, layer 2의 가중치
    """
    self.inputs = 2
    self.hiddenNodes = 3
    self.outputNodes = 1
    
    # 가중치를 초기화 합니다.
    # layer 1 가중치 shape : 2x3
    self.w1 = np.random.randn(self.inputs,self.hiddenNodes)
    
    # layer 2 가중치 shape : 3x1
    self.w2 = np.random.randn(self.hiddenNodes, self.outputNodes)

## 활성화 함수 구현
입력값의 시그모이드 함수의 결과값 

In [101]:
class NN2(NN1) :

  def __init__(self) :
    super().__init__()

  def sigmoid(self, s):
    """
    활성화 함수인 시그모이드 함수를 정의합니다.
    s : 활성화 함수에 입력되는 값(=가중합)
    """
    return 1 / (1+np.exp(-s))

## 순전파
weight, bias 의 가중합 / 활성화 함수를 사용하여 입력층에서 은닉층을 거쳐, 출력층까지 계산.

### 입력층
데이터셋이 입력되는 층으로 features 수 만큼의 입력 노드를 갖는다.

### 은닉층
입력 신호를 가중합(가중치, 편향)하여 처리하는 층으로, 사용자가 볼 수 없으며, 노드수는 자유롭게 결정 가능하다. 2개 이상의 은닉층을 가진 신경망을 `딥러닝`이라고 한다.

### 출력층
연산을 마친 값이 출력되는 층이다. 문제의 종류에 따라, 노드 수, 활성화 함수가 결정된다.

#### 이진 분류
활성화 함수는 시그모이드, 노드 수 1개, 0~1 사이의 확률값이 출력

#### 다중 분류
활성화 함수는 소프트맥스, 노드 수는 레이블릐 클래스 수와 동일. 각 클래스일 확률을 0~1 사이 값으로 출력하며, 합은 1이 된다.

#### 회귀
활성화 함수는 사용하지 않으며, 출력층의 노드 수는 출력값의 특성 수와 동일.

In [102]:
class NN3(NN2) :

  def __init__(self) :
    super().__init__()

  def feed_forward(self, X):
    """
    순전파를 구현합니다.
    입력 신호를 받아 출력층의 결과를 반환합니다.

    hidden_sum : 은닉층(layer 1)에서의 가중합(weighted sum)
    activated_hidden : 은닉층(layer 1) 활성화 함수의 함숫값
    output_sum : 출력층(layer 2)에서의 가중합(weighted sum)
    activated_output : 출력층(layer 2) 활성화 함수의 함숫값
    """

    self.hidden_sum = np.dot(X, self.w1)
    self.activated_hidden = self.sigmoid(self.hidden_sum)

    self.output_sum = np.dot(self.activated_hidden, self.w2)
    self.activated_output = self.sigmoid(self.output_sum)

    return self.activated_output

## 활성화 함수의 도함수 구현
sigmoid를 미분한 함수 

In [103]:
class NN4(NN3) :

  def __init__(self) :
    super().__init__()

  def sigmoidPrime(self, s):
    """
    활성화 함수(sigmoid)를 미분한 함수입니다.
    s : 순전파 과정에서 활성화 함수에 입력되는 값(=가중합)
    """
    sx = self.sigmoid(s)
    return sx * (1-sx)

## 역전파
출력층에서 손실 값(Error)를 구한 뒤에 이를 각 가중치에 대해 미분한 값만큼 가중치를 수정합니다.


In [104]:
class NN5(NN4) :

  def __init__(self) :
    super().__init__()

  def backward(self, X, y, o):
    """
    역전파를 구현합니다.
    출력층에서 손실 값(Error)를 구한 뒤에 이를 각 가중치에 대해 미분한 값만큼 가중치를 수정합니다.

    X : 입력 데이터(input)
    y : 타겟값(target value)
    o : 출력값(output)

    o_error : 손실(Error) = 타겟값과 출력값의 차이
    o_delta : 출력층 활성화 함수의 미분값
    """

    # o_error : 손실(Error)을 구합니다.
    self.o_error = y - o 

    # o_delta : 활성화 함수(시그모이드)의 도함수를 사용하여 출력층 활성화 함수 이전의 미분값을 구합니다.
    self.o_delta = self.o_error * self.sigmoidPrime(o)

    # z2 error : 은닉층에서의 손실을 구합니다.
    self.z2_error = self.o_delta.dot(self.w2.T)

    # z2 delta : 활성화 함수(시그모이드)의 도함수를 사용하여 은닉층 활성화 함수 이전의 미분값을 구합니다.
    self.z2_delta = self.z2_error*self.sigmoidPrime(self.output_sum)

    # w1, w2를 업데이트 합니다.
    self.w1 += X.T.dot(self.z2_delta) # X * dE/dY * dY/dy(=Y(1-Y))
    self.w2 += self.activated_hidden.T.dot(self.o_delta) # H1 * Y(1-Y) * (Y - o)


## 신경망 학습
순전파와 역전파를 반복하며, 가중치 갱신

In [105]:
class NN6(NN5) :

  def __init__(self) :
    super().__init__()
  
  def train(self, X, y):
    """
    실제로 신경망 학습을 진행하는 코드입니다.
    1번의 순전파-역전파, 즉 1 iteration 을 수행하는 함수입니다.
    
    X : 입력 데이터(input)
    y : 타겟값(target value)
    """
    o = self.feed_forward(X)
    self.backward(X,y,o)

# 구현한 신경망 실행

## 샘플 데이터 생성

In [106]:
# [공부시간, 수면시간]
X = np.array(([8,8],
              [2,5],
              [7,6]), dtype=float)

# 선형 관계를 바탕으로 시험 점수 레이블을 생성합니다.
y = X[:,0]*5 + X[:,1]*2
y = y.reshape(3,1)

## 샘플 데이터 정규화

In [107]:
X = X / np.amax(X, axis=0)
y = y / np.amax(y, axis=0)

print("공부시간, 수면시간 \n", X)
print("시험점수 \n", y)

공부시간, 수면시간 
 [[1.    1.   ]
 [0.25  0.625]
 [0.875 0.75 ]]
시험점수 
 [[1.        ]
 [0.35714286]
 [0.83928571]]


## 신경망 학습

## 신경망 인스턴스 생성

In [108]:
nn = NN6()

2022.12.02 08:27:40 __main__ 신경망 초기화


In [109]:
# 반복수(epochs or iterations)를 정합니다.
iter = 10000

# 지정한 반복수 만큼 반복합니다.
for i in range(iter):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 1000 == 0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('입력: \n', X)
        print('타겟출력: \n', y)
        print('예측: \n', str(nn.feed_forward(X)))
        print("에러: \n", str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X,y)

+---------EPOCH 1---------+
입력: 
 [[1.    1.   ]
 [0.25  0.625]
 [0.875 0.75 ]]
타겟출력: 
 [[1.        ]
 [0.35714286]
 [0.83928571]]
예측: 
 [[0.56611111]
 [0.50635471]
 [0.5515955 ]]
에러: 
 0.09776313366577666
+---------EPOCH 2---------+
입력: 
 [[1.    1.   ]
 [0.25  0.625]
 [0.875 0.75 ]]
타겟출력: 
 [[1.        ]
 [0.35714286]
 [0.83928571]]
예측: 
 [[0.59499499]
 [0.53175447]
 [0.57987284]]
에러: 
 0.08727110418086405
+---------EPOCH 3---------+
입력: 
 [[1.    1.   ]
 [0.25  0.625]
 [0.875 0.75 ]]
타겟출력: 
 [[1.        ]
 [0.35714286]
 [0.83928571]]
예측: 
 [[0.61954945]
 [0.55346073]
 [0.60397965]]
에러: 
 0.07955075801894661
+---------EPOCH 4---------+
입력: 
 [[1.    1.   ]
 [0.25  0.625]
 [0.875 0.75 ]]
타겟출력: 
 [[1.        ]
 [0.35714286]
 [0.83928571]]
예측: 
 [[0.64049957]
 [0.57204728]
 [0.62459987]]
에러: 
 0.07383816030092423
+---------EPOCH 5---------+
입력: 
 [[1.    1.   ]
 [0.25  0.625]
 [0.875 0.75 ]]
타겟출력: 
 [[1.        ]
 [0.35714286]
 [0.83928571]]
예측: 
 [[0.65843347]
 [0.58799003]
 [0.6422917

In [110]:
# 각각의 변수(가중치)를 디스플레이 하기 위한 코드입니다.
attributes = ['w1', 'hidden_sum', 'activated_hidden', 'w2', 'activated_output']

for i in attributes:
    if i[:2] != '__':
        print(i+'\n', getattr(nn,i), '\n'+'---'*3)

w1
 [[-0.53830378 -0.37420207  1.71467012]
 [ 1.24627641 -0.56769715 -1.0470542 ]] 
---------
hidden_sum
 [[ 0.70795724 -0.94198675  0.66770378]
 [ 0.64433816 -0.44841041 -0.22569198]
 [ 0.46367963 -0.75326716  0.71511345]] 
---------
activated_hidden
 [[0.66994963 0.2804992  0.66098881]
 [0.65573345 0.38973877 0.44381529]
 [0.61388672 0.32010982 0.67153006]] 
---------
w2
 [[-1.64131314]
 [-9.33397483]
 [ 9.36921135]] 
---------
activated_output
 [[0.92235766]
 [0.36444961]
 [0.90854076]] 
---------
